## ISBN 분류 알아보기

#### 9 7 8 - 8 9 - 9 4 5 6 7 - 6 7 - 9 
1) 978 -> 접두어 (978 또는 979)
2) 89 -> 출판 국가 또는 언어 번호
3) 94567 -> 출판사 번호
4) 67 -> 항목 번호
5) 9 -> 체크 합계 번호

In [2]:
import pandas as pd
lib = pd.read_csv('2023_01.csv')

C:\Users\shs21\AppData\Local\Temp\ipykernel_20248\3806304882.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lib = pd.read_csv('2023_01.csv')


In [4]:
# 주제분류번호, ISBN을 문자열로 변환
lib['주제분류번호'] = lib['주제분류번호'].astype(str)  # float를 문자열로 반환
lib['ISBN'] = lib['ISBN'].astype(str)
lib

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1.0,"헤이세이사 - 1989-2019 어제의 세계, 모든 것","요나하 준 지음, 이충원 옮김",마르코폴로,2022,9791192667102.0,NaN,NaN,NaN,nan,1.0,0.0,2023-01-30
1,2.0,몸으로 읽는 세계사 - 사소한 몸에 숨겨진 독특하고 거대한 문명의 역사,"캐스린 페트라스, 로스 페트라스 (지은이), 박지선 (옮긴이)",다산초당(다산북스),2023,9791130695303.0,NaN,NaN,NaN,nan,1.0,0.0,2023-01-30
2,3.0,히틀러의 주치의들 - 권력자들의 삶과 죽음,양성관 (지은이),드러커마인드,2023,9791191384376.0,NaN,NaN,NaN,nan,1.0,0.0,2023-01-30
3,4.0,"우리의 기원, 단일하든 다채롭든 - 상상과 과학의 경계에서 찾아가는 한민족의 흔적",강인욱 지음,21세기북스,2022,9788950978877.0,NaN,NaN,NaN,nan,1.0,0.0,2023-01-30
4,5.0,광화문 괴담 - 오류와 왜곡에 맞서는 박종인 기자의 역사 전쟁,박종인 (지은이),와이즈맵,2022,9791189328986.0,NaN,NaN,NaN,nan,1.0,0.0,2023-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
429767,385433.0,떨어진 꽃은 줍지 않는다,김정섭 지음,중앙일보사,1994,9788946100800,NaN,NaN,4,813.6,1.0,0.0,1970-01-01
429768,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN
429769,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN
429770,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN


## 1) 한국 출판 책인지, 아닌지 판별하기

In [65]:
# ISBN 열만 선택하여 데이터 전처리
isbn_lib = lib['ISBN'].dropna().astype(str)

# 출판 국가를 판별하는 함수 정의
def get_country_from_isbn(isbn):
    if isbn.startswith('97889') or isbn.startswith('97911') or isbn.startswith('97912'):
        return 'South Korea'
    elif isbn.startswith('978'):
        return 'Other'
    elif isbn.startswith('979'):
        return 'Other'
    else:
        return 'Unknown'

# 각 ISBN에 대해 출판 국가를 판별
isbn_lib_country = isbn_lib.apply(get_country_from_isbn)

# 출판 국가별 도서 수 집계
country_counts = isbn_lib_country.value_counts().head(10)

# 결과를 표시
print(country_counts)

ISBN
South Korea    365010
Unknown         48638
Other           16124
Name: count, dtype: int64


## 2) 외국어로 된 책 중에서, 가장 많이 사용되는 언어 5개

### [1] 직접 단순 if문으로 대략적인 조사

In [73]:
# "Other"로 분류된 ISBN에서 출판 국가를 더 구체적으로 분석

# 먼저, "Other"로 분류된 ISBN만 필터링
other_isbns = isbn_lib[isbn_lib_country == 'Other']

# 출판 국가를 더 구체적으로 판별하는 함수
def get_more_specific_country_from_isbn(isbn):
    if isbn.startswith('9780') or isbn.startswith('9781'):
        return 'English-speaking Area'
    elif isbn.startswith('9782'):
        return 'French-speaking Area'
    elif isbn.startswith('9783'):
        return 'German-speaking Area'
    elif isbn.startswith('9784'):
        return 'Japan'
    elif isbn.startswith('9785'):
        return 'Russian-speaking Area'
    elif isbn.startswith('9787'):
        return 'China'
    elif isbn.startswith('97880'):
        return 'Czech Republic'
    elif isbn.startswith('97882'):
        return 'Norway'
    else:
        return 'Other'

# 각 ISBN에 대해 더 구체적인 출판 국가를 판별
more_specific_country_data = other_isbns.apply(get_more_specific_country_from_isbn)

# 출판 국가별 도서 수 집계
more_specific_country_counts = more_specific_country_data.value_counts().head(6)

# 결과를 표시
print(more_specific_country_counts)

ISBN
English-speaking Area    8288
Other                    3088
Japan                    3081
China                    1486
French-speaking Area       63
German-speaking Area       59
Name: count, dtype: int64


### [2] json 파일 연결로 상세 조사

In [85]:
import json

# JSON 파일 경로를 지정합니다.
json_file_path = 'isbn_country_mapping.json'

# JSON 파일을 불러옵니다.
with open(json_file_path, 'r', encoding='utf-8') as file:
    isbn_country_mapping = json.load(file)

# ISBN에서 국가를 추출하는 함수 정의
def get_country_from_isbn(isbn):
    for prefix, country in isbn_country_mapping.items():
        if isbn.startswith('978' + prefix) or isbn.startswith('979' + prefix):
            return country
    return 'Unknown'

# ISBN 열만 선택하여 데이터 전처리
isbn_lib = lib['ISBN'].dropna().astype(str)

# 각 ISBN에 대해 출판 국가를 판별
isbn_lib_country = isbn_lib.apply(get_country_from_isbn)

# 'Unknown' 국가 제외
isbn_lib_country = isbn_lib_country[isbn_lib_country != 'Unknown']

# 출판 국가별 도서 수 집계
country_counts = isbn_lib_country.value_counts()

# 결과를 표시
print(country_counts)

ISBN
South Korea                     301647
English-speaking Area            71681
Japan                             3081
China                             1488
Thailand                          1077
Philippines                        113
Indonesia                           91
Italy                               85
Albania                             70
French-speaking Area                63
Netherlands                         61
Mongolia                            61
German-speaking Area                59
Russian-speaking Area               54
Nigeria                             44
Singapore                           43
Spain                               35
Sweden                              35
Taiwan                              34
International NGO Publishers        23
India                               22
Hong Kong                           18
Greece                              16
Malaysia                            12
Argentina                            6
Bulgaria            